## Cloud storage

In [ ]:
def solution(queries):
    cloud_storage = {}
    result = []
    users = {}

    users["admin"]={"capacity": float("inf"), "files": cloud_storage}

    def add_file(cloud_storage, name, size):
        if name in cloud_storage:
            return "false"
        cloud_storage[name] = int(size)
        return "true"

    def copy_file(users, name_from, name_to):

        for user_id, user in users.items():
            if name_from in user["files"] and "/" != name_from[:-1]:
                file_size = user["files"][name_from]
                if (
                    name_to not in user["files"]
                    and (user_id == "admin" or users[user_id]["capacity"] >= file_size)
                ):
                    user["files"][name_to] = file_size
                    if user_id != "admin":
                        users[user_id]["capacity"] -= file_size
                    return "true"
        return "false"

        # if name_from not in cloud_storage or "/" == name_from[:-1] or name_to in cloud_storage or "/" == name_from[:-1]:

        #     return "false"
        # cloud_storage[name_to] = {"size": cloud_storage[name_from]["size"]}
        # return "true"

    def get_file_size(cloud_storage, name):
        for user_id, user in users.items():
            file_size = user["files"].get(name)
            if file_size:
                if user_id == "admin":
                    return str(cloud_storage.get(name, {}))
                else:
                    return str(file_size)
        return ""
    
    def find_file(prefix, suffix):
        matching_files = []

        for user_id, user in users.items():
            for name, file_info in user["files"].items():
                if name.startswith(prefix) and name.endswith(suffix):
                    matching_files.append((name, file_info))

        if not matching_files:
            return ""

        # Sort files in descending order of size, and lexicographically in case of ties
        sorted_files = sorted(matching_files, key=lambda x: (-x[1], x[0]), reverse=False)
        

        # Format the result string
        result_str = ", ".join(f"{name}({size})" for name, size in sorted_files)
        return result_str
    
    def add_user(users, user_id, capacity):
        if user_id in users:
            return "false"
        users[user_id] = {"capacity": int(capacity), "files": {}}
        return "true"
    def add_file_by(users, user_id, name, size):

        if user_id not in users or name in users[user_id]["files"] or users[user_id]["capacity"] < int(size) or any(name in user["files"] for user in users.values()):
            return ""

        users[user_id]["files"][name] = int(size)
        users[user_id]["capacity"] -= int(size)

        return str(users[user_id]["capacity"])

    def update_capacity(users, user_id, new_capacity):
            if user_id not in users:
                return ""

            # Remove files if the new capacity is exceeded
            current_capacity = users[user_id]["capacity"]
            new_capacity = int(new_capacity)
            
            total_size = sum(users[user_id]["files"].values())

            if total_size > new_capacity:
                files_to_remove = 0
                remaining_capacity = new_capacity
                files_list = list(users[user_id]["files"].items())

                # Sort files lexicographically and by size
                sorted_files = sorted(users[user_id]["files"].items(), key=lambda x: (-x[1], x[0]), reverse=False)
                sorted_files.reverse()
                print("sorted files",sorted_files)
                while total_size > new_capacity:
                    popfile = sorted_files.pop()
                    removedfile = users[user_id]["files"].pop(popfile[0])
                    # print("ATTENTION 888",removedfile,users[user_id]["files"])

                    total_size -= removedfile
                    files_to_remove+=1
                    # print("removed file", popfile, removedfile)
                    # current_capacity += removedfile
                    # print("removing file",user_id, users[user_id]["files"], current_capacity)
                    
                # for file, size in sorted_files:
                #     if remaining_capacity >= size:
                #         remaining_capacity -= size
                #     else:
                #         print("FILES TO REMOVE", file, size)
                #         files_to_remove.append(file )

                # Remove files from user's storage
                
                # for file in files_to_remove:
                #     removedfile = users[user_id]["files"].pop(file)
                #     print("removed file", removedfile)
                #     current_capacity += removedfile
                #     print("removing file",user_id, users[user_id]["files"], current_capacity)
                
                users[user_id]["capacity"] = new_capacity - total_size
                return str(files_to_remove)
            
            users[user_id]["capacity"] = new_capacity - total_size
            return "0"
    def compress_file(users, user_id, name):
        if not name.endswith(".COMPRESSED") and name in users[user_id]["files"]:
            original_size = users[user_id]["files"][name]
            compressed_size = original_size // 2

            if users[user_id]["capacity"] >= compressed_size:
                compressed_name = name + ".COMPRESSED"
                users[user_id]["files"][compressed_name] = compressed_size
                users[user_id]["capacity"] += compressed_size
                del users[user_id]["files"][name]
                return str(users[user_id]["capacity"])

        return ""

    def decompress_file(users, user_id, name):
        if name.endswith(".COMPRESSED") and name in users[user_id]["files"]:
            decompressed_name = name.replace(".COMPRESSED", "")
            decompressed_size = users[user_id]["files"][name] * 2
            if decompressed_name not in users[user_id]["files"] and users[user_id]["capacity"] >= decompressed_size:
                users[user_id]["files"][decompressed_name] = decompressed_size
                users[user_id]["capacity"] -= decompressed_size
                #FIX HERE, delete the file[name] in users[user_id]["files"]

                return str(users[user_id]["capacity"])
        return ""
    #MAIN
    for query in queries:
        operation, *params = query
        if operation == "ADD_FILE":
            result.append(add_file(cloud_storage, *params))
        elif operation == "COPY_FILE":
            result.append(copy_file(users, *params))
        elif operation == "GET_FILE_SIZE":
            result.append(get_file_size(cloud_storage, *params))
        elif operation == "FIND_FILE":
            result.append(find_file(*params))
        elif operation == "ADD_USER":
            result.append(add_user(users, *params))
        elif operation == "ADD_FILE_BY":
            result.append(add_file_by(users, *params))
        elif operation == "UPDATE_CAPACITY":
            result.append(update_capacity(users, *params))
        elif operation == "COMPRESS_FILE":
            result.append(compress_file(users, *params))
        elif operation == "DECOMPRESS_FILE":
            result.append(decompress_file(users, *params))
    return result




# queries = [["ADD_USER","owner","1000"], 
#  ["ADD_FILE_BY","owner","/foo/bar/large_file","600"], 
#  ["ADD_FILE_BY","owner","/foo/small_file","200"], 
#  ["ADD_FILE_BY","owner","/foo/medium_file","400"], 
#  ["UPDATE_CAPACITY","owner","1500"], 
#  ["UPDATE_CAPACITY","owner","800"], 
#  ["UPDATE_CAPACITY","owner","500"], 
#  ["UPDATE_CAPACITY","owner","100"]]
# ex = ["true", "400", "200", "", "0", "0", "1", "1"]
# print(solution(queries))

## Banking

In [ ]:
def solution(queries):
    accounts = {}
    transfers = {}
    transfer_counter = 0
    MILLISECONDS_IN_1_DAY = 86400000


    def create_account(timestamp, account_id):
        if account_id not in accounts:
            accounts[account_id] = {"balance": 0, "transactions": [], "transfers": {}}
            return "true"
        else:
            return "false"

    def deposit(timestamp, account_id, amount):
        if account_id in accounts:
            amount = int(amount)
            
            # if accounts[account_id]: 
            #     if accounts[account_id]["balance"]:
            #         print( accounts[account_id]["balance"] )

            accounts[account_id]["balance"] += amount
            accounts[account_id]["transactions"].append({"type": "DEPOSIT", "amount": amount, "timestamp": int(timestamp)})
            
            
            return str(accounts[account_id]["balance"])
        else:
            return ""

    def pay(timestamp, account_id, amount):
        if account_id in accounts:
            amount = int(amount)
            if accounts[account_id]["balance"] >= amount:
                accounts[account_id]["balance"] -= amount
                accounts[account_id]["transactions"].append({"type": "PAY", "amount": amount, "timestamp": int(timestamp)})
                return str(accounts[account_id]["balance"])
            else:
                return ""
        else:
            return ""
    
    def top_activity(timestamp, n):
            sorted_accounts = sorted(accounts.items(), key=lambda x: (-sum(t['amount'] for t in x[1]['transactions']), x[0]), reverse=False)
            
            top_n_accounts = sorted_accounts[:n]
            result = ", ".join(f"{account_id}({sum(t['amount'] for t in account_data['transactions'])})" for account_id, account_data in top_n_accounts)
            return result

    def transfer(timestamp, source_account_id, target_account_id, amount):
        nonlocal transfer_counter
        if source_account_id == target_account_id or source_account_id not in accounts or target_account_id not in accounts:
            return ""

        amount = int(amount)
        if accounts[source_account_id]["balance"] < amount:
            return ""

        transfer_id = f"transfer{transfer_counter + 1}"
        transfer_counter += 1

        transfers[transfer_id] = {
            "source_account": source_account_id,
            "target_account": target_account_id,
            "amount": amount,
            "timestamp": int(timestamp),
            "accepted": False
        }

        accounts[source_account_id]["balance"] -= amount
        accounts[source_account_id]["transfers"][transfer_id] = {"type": "TRANSFER_OUT", "amount": amount, "timestamp": int(timestamp)}

        accounts[target_account_id]["transfers"][transfer_id] = {"type": "TRANSFER_IN", "amount": amount, "timestamp": int(timestamp)}

        return transfer_id
    
    def accept_transfer(timestamp, account_id, transfer_id):
      
        if (transfer_id not in transfers 
        or transfers[transfer_id]["accepted"] 
        #expired
        or int(timestamp) > transfers[transfer_id]["timestamp"] + MILLISECONDS_IN_1_DAY  
        
        or account_id != transfers[transfer_id]["target_account"]):
            if transfer_id in transfers:
                if int(timestamp) > transfers[transfer_id]["timestamp"] + MILLISECONDS_IN_1_DAY:
                    
                    source_account_id = transfers[transfer_id]["source_account"]
                    amount = transfers[transfer_id]["amount"]
                    accounts[source_account_id]["balance"] += amount

            
            return "false"
        else:
            source_account_id = transfers[transfer_id]["source_account"]
            
            
            
            
            amount = transfers[transfer_id]["amount"]

            transfers[transfer_id]["accepted"] = True
            
            accounts[account_id]["balance"] += amount
            
            accounts[account_id]["transactions"].append({"type": "TRANSFER_IN", "amount": amount, "timestamp": int(timestamp)})
            #? not sure + or - for out amount
            accounts[source_account_id]["transactions"].append({"type": "TRANSFER_OUT", "amount": amount, "timestamp": int(timestamp)})

            return "true"
    
    
    #MAIN
    output = []
    
    for query in queries:
        operation, timestamp, account_id, *args = query
        if operation == "CREATE_ACCOUNT":
            output.append(create_account(timestamp, account_id))
        elif operation == "DEPOSIT":
            output.append(deposit(timestamp, account_id, *args))
        elif operation == "PAY":
            output.append(pay(timestamp, account_id, *args))
        elif operation == "TOP_ACTIVITY":
            output.append(top_activity(timestamp, int(account_id)))
        elif operation == "TRANSFER":
            output.append(transfer(timestamp, account_id, *args))
        elif operation == "ACCEPT_TRANSFER":
            output.append(accept_transfer(timestamp, account_id, *args))
    return output


queries = [
      [
        "CREATE_ACCOUNT",
        "1",
        "acc0"
      ],
      [
        "CREATE_ACCOUNT",
        "2",
        "acc1"
      ],
      [
        "CREATE_ACCOUNT",
        "3",
        "acc2"
      ],
      [
        "CREATE_ACCOUNT",
        "4",
        "acc3"
      ],
      [
        "CREATE_ACCOUNT",
        "5",
        "acc4"
      ],
      [
        "CREATE_ACCOUNT",
        "6",
        "acc5"
      ],
      [
        "CREATE_ACCOUNT",
        "7",
        "acc6"
      ],
      [
        "CREATE_ACCOUNT",
        "8",
        "acc7"
      ],
      [
        "CREATE_ACCOUNT",
        "9",
        "acc8"
      ],
      [
        "CREATE_ACCOUNT",
        "10",
        "acc9"
      ],
      [
        "DEPOSIT",
        "11",
        "acc0",
        "7757"
      ],
      [
        "DEPOSIT",
        "12",
        "acc1",
        "8114"
      ],
      [
        "DEPOSIT",
        "13",
        "acc2",
        "6692"
      ],
      [
        "DEPOSIT",
        "14",
        "acc3",
        "5429"
      ],
      [
        "DEPOSIT",
        "15",
        "acc4",
        "7852"
      ],
      [
        "DEPOSIT",
        "16",
        "acc5",
        "6105"
      ],
      [
        "DEPOSIT",
        "17",
        "acc6",
        "5747"
      ],
      [
        "DEPOSIT",
        "18",
        "acc7",
        "8009"
      ],
      [
        "DEPOSIT",
        "19",
        "acc8",
        "5165"
      ],
      [
        "DEPOSIT",
        "20",
        "acc9",
        "5966"
      ],
      [
        "PAY",
        "21",
        "acc0",
        "344"
      ],
      [
        "PAY",
        "22",
        "acc1",
        "222"
      ],
      [
        "PAY",
        "23",
        "acc2",
        "377"
      ],
      [
        "PAY",
        "24",
        "acc3",
        "172"
      ],
      [
        "PAY",
        "25",
        "acc4",
        "251"
      ],
      [
        "PAY",
        "26",
        "acc5",
        "497"
      ],
      [
        "PAY",
        "27",
        "acc6",
        "472"
      ],
      [
        "PAY",
        "28",
        "acc7",
        "103"
      ],
      [
        "PAY",
        "29",
        "acc8",
        "171"
      ],
      [
        "PAY",
        "30",
        "acc9",
        "448"
      ],
      [
        "TRANSFER",
        "31",
        "acc6",
        "acc0",
        "1358"
      ],
      [
        "TRANSFER",
        "32",
        "acc0",
        "acc1",
        "1150"
      ],
      [
        "TRANSFER",
        "33",
        "acc3",
        "acc2",
        "1235"
      ],
      [
        "TRANSFER",
        "34",
        "acc0",
        "acc3",
        "1539"
      ],
      [
        "TRANSFER",
        "35",
        "acc2",
        "acc4",
        "1253"
      ],
      [
        "TRANSFER",
        "36",
        "acc2",
        "acc5",
        "1397"
      ],
      [
        "TRANSFER",
        "37",
        "acc5",
        "acc6",
        "1861"
      ],
      [
        "TRANSFER",
        "38",
        "acc2",
        "acc7",
        "1518"
      ],
      [
        "TRANSFER",
        "39",
        "acc3",
        "acc8",
        "1635"
      ],
      [
        "TRANSFER",
        "40",
        "acc1",
        "acc9",
        "1669"
      ],
      [
        "TOP_ACTIVITY",
        "41",
        "10"
      ],
      [
        "DEPOSIT",
        "86400041",
        "acc0",
        "506"
      ],
      [
        "DEPOSIT",
        "86400042",
        "acc1",
        "276"
      ],
      [
        "DEPOSIT",
        "86400043",
        "acc2",
        "361"
      ],
      [
        "DEPOSIT",
        "86400044",
        "acc3",
        "757"
      ],
      [
        "DEPOSIT",
        "86400045",
        "acc4",
        "129"
      ],
      [
        "DEPOSIT",
        "86400046",
        "acc5",
        "477"
      ],
      [
        "DEPOSIT",
        "86400047",
        "acc6",
        "676"
      ],
      [
        "DEPOSIT",
        "86400048",
        "acc7",
        "754"
      ],
      [
        "DEPOSIT",
        "86400049",
        "acc8",
        "873"
      ],
      [
        "DEPOSIT",
        "86400050",
        "acc9",
        "242"
      ],
      [
        "ACCEPT_TRANSFER",
        "86400051",
        "acc0",
        "transfer1"
      ],
      [
        "ACCEPT_TRANSFER",
        "86400052",
        "acc1",
        "transfer2"
      ],
      [
        "ACCEPT_TRANSFER",
        "86400053",
        "acc2",
        "transfer3"
      ],
      [
        "ACCEPT_TRANSFER",
        "86400054",
        "acc3",
        "transfer4"
      ],
      [
        "ACCEPT_TRANSFER",
        "86400055",
        "acc4",
        "transfer5"
      ],
      [
        "ACCEPT_TRANSFER",
        "86400056",
        "acc5",
        "transfer6"
      ],
      [
        "ACCEPT_TRANSFER",
        "86400057",
        "acc6",
        "transfer7"
      ],
      [
        "ACCEPT_TRANSFER",
        "86400058",
        "acc7",
        "transfer8"
      ],
      [
        "ACCEPT_TRANSFER",
        "86400059",
        "acc8",
        "transfer9"
      ],
      [
        "ACCEPT_TRANSFER",
        "86400060",
        "acc9",
        "transfer10"
      ],
      [
        "TOP_ACTIVITY",
        "86400061",
        "10"
      ],
      [
        "PAY",
        "86400062",
        "acc0",
        "348"
      ],
      [
        "PAY",
        "86400063",
        "acc1",
        "166"
      ],
      [
        "PAY",
        "86400064",
        "acc2",
        "281"
      ],
      [
        "PAY",
        "86400065",
        "acc3",
        "267"
      ],
      [
        "PAY",
        "86400066",
        "acc4",
        "421"
      ],
      [
        "PAY",
        "86400067",
        "acc5",
        "387"
      ],
      [
        "PAY",
        "86400068",
        "acc6",
        "172"
      ],
      [
        "PAY",
        "86400069",
        "acc7",
        "427"
      ],
      [
        "PAY",
        "86400070",
        "acc8",
        "481"
      ],
      [
        "PAY",
        "86400071",
        "acc9",
        "459"
      ],
      [
        "TRANSFER",
        "86400073",
        "acc9",
        "acc0",
        "1938"
      ],
      [
        "TRANSFER",
        "86400074",
        "acc3",
        "acc1",
        "1764"
      ],
      [
        "TRANSFER",
        "86400075",
        "acc7",
        "acc2",
        "1660"
      ],
      [
        "TRANSFER",
        "86400076",
        "acc4",
        "acc3",
        "1541"
      ],
      [
        "TRANSFER",
        "86400077",
        "acc8",
        "acc4",
        "1379"
      ],
      [
        "TRANSFER",
        "86400078",
        "acc2",
        "acc5",
        "1219"
      ],
      [
        "TRANSFER",
        "86400079",
        "acc5",
        "acc6",
        "1519"
      ],
      [
        "TRANSFER",
        "86400080",
        "acc2",
        "acc7",
        "1832"
      ],
      [
        "TRANSFER",
        "86400081",
        "acc6",
        "acc8",
        "1645"
      ],
      [
        "TRANSFER",
        "86400082",
        "acc2",
        "acc9",
        "1960"
      ],
      [
        "ACCEPT_TRANSFER",
        "172800083",
        "acc0",
        "transfer11"
      ],
      [
        "ACCEPT_TRANSFER",
        "172800084",
        "acc1",
        "transfer12"
      ],
      [
        "ACCEPT_TRANSFER",
        "172800085",
        "acc2",
        "transfer13"
      ],
      [
        "ACCEPT_TRANSFER",
        "172800086",
        "acc3",
        "transfer14"
      ],
      [
        "ACCEPT_TRANSFER",
        "172800087",
        "acc4",
        "transfer15"
      ],
      [
        "ACCEPT_TRANSFER",
        "172800088",
        "acc5",
        "transfer16"
      ],
      [
        "ACCEPT_TRANSFER",
        "172800089",
        "acc6",
        "transfer17"
      ],
      [
        "ACCEPT_TRANSFER",
        "172800090",
        "acc7",
        "transfer18"
      ],
      [
        "ACCEPT_TRANSFER",
        "172800091",
        "acc8",
        "transfer19"
      ],
      [
        "ACCEPT_TRANSFER",
        "172800092",
        "acc9",
        "transfer20"
      ],
      [
        "TOP_ACTIVITY",
        "172800094",
        "10"
      ]
    ]

print(solution(queries))

## Locker

In [ ]:
def solution(queries):
    in_memory_db = {}
    modification_count = {}
    lock_queue = {}
    lock_status = {}
    def set_or_inc(key, field, value):
        #debug
        if get_key_locked_status(key):
            print("setorinc ignored", key)
        if not get_key_locked_status(key):
            if key not in in_memory_db:
                in_memory_db[key] = {field: int(value)}
            elif field not in in_memory_db[key]:
                in_memory_db[key][field] = int(value)
            else:
                in_memory_db[key][field] += int(value)

            # Update modification count
            modification_count[key] = modification_count.get(key, 0) + 1

        return str(in_memory_db[key][field])

    def get(key, field):
        if key in in_memory_db and field in in_memory_db[key]:
            return str(in_memory_db[key][field])
        else:
            return ""

    def delete(key, field):
        if key in in_memory_db and field in in_memory_db[key] and not get_key_locked_status(key):
            del in_memory_db[key][field]
            modification_count[key]+=1

            # If all fields in a record have been deleted, remove the record
            if not in_memory_db[key]:
                del in_memory_db[key]
                del modification_count[key]

            return "true"
        else:
            return "false"

    def top_n_keys(n):
        n = int(n)
        sorted_keys = sorted(
            modification_count.keys(),
            #if tie, sort by k (key names)lexicographical 
            key=lambda k: (-modification_count[k], k),
            reverse=False,
        )

        result = []

        for key in sorted_keys[:n]:
            result.append(f"{key}({modification_count[key]})")

        return ", ".join(result)
    def get_key_locked_status(key) -> bool:
        return key in lock_status
    def set_or_inc_by_caller(key, field, value, caller_id):
        if key not in in_memory_db or not get_key_locked_status(key) or lock_status[key]["user"] == caller_id:
            if key in lock_status:
                if lock_status[key]["user"] == caller_id:
                    print(key, field, value, caller_id)
            return set_or_inc(key, field, value)
        else:
            return str(in_memory_db[key].get(field, ""))if field in in_memory_db[key] else ""

    def delete_by_caller(key, field, caller_id):
        if key in in_memory_db and (not get_key_locked_status(key) or lock_status[key]["user"] == caller_id):
            return delete(key, field)
        else:
            return "false"

    def lock(caller_id, key):
        if key not in in_memory_db:
            return "invalid_request"
        #  if the record is already locked by another user
        if get_key_locked_status(key):
            if lock_status[key]["user"] == caller_id or caller_id in lock_queue[key]:
                return ""
            elif caller_id not in lock_queue[key]:
                lock_queue[key].append(caller_id)
                return "wait"
            else:
                return ""
        else:
            lock_queue[key] = []
            lock_status[key] = {"user": caller_id, "queue": lock_queue[key]}
            return "acquired"

    def unlock(key):
        if key in lock_status:
            released_user = lock_status[key]["user"]
            lock_queue[key] = lock_status[key]["queue"]
            del lock_status[key]
            return "released" if released_user else ""
        else:
            return "invalid_request"


    result = []

    for query in queries:
        operation, *params = query
        if operation == "SET_OR_INC":
            result.append(set_or_inc(*params))
        elif operation == "UNLOCK":
            result.append(unlock(*params))
        elif operation == "LOCK":
            result.append(lock(*params))
        elif operation == "DELETE":
            result.append(delete(*params))
        elif operation == "TOP_N_KEYS":
            result.append(top_n_keys(*params))
        elif operation == "SET_OR_INC_BY_CALLER":
            result.append(set_or_inc_by_caller(*params))
        elif operation == "DELETE_BY_CALLER":
            result.append(delete_by_caller(*params))
        elif operation == "GET":
            result.append(get(*params))

    return result


# # Example Usage
# queries = [
#     ["SET_OR_INC", "C", "field1", "10"],
#     ["TOP_N_KEYS", "5"],
#     ["SET_OR_INC", "A", "
# leet4.py
# queries:
# [["TOP_N_KEYS","10"], 
#  ["GET","dept4","floors"], 
#  ["SET_OR_INC_BY_CALLER","dept4","floors","10","builder"], 
#  ["SET_OR_INC_BY_CALLER","dept4","floors","4","otherBuilder"], 
#  ["SET_OR_INC","dept3","floors","6"], 
#  ["SET_OR_INC_BY_CALLER","dept2","floors","10","builder"], 
#  ["GET","dept4","floors"], 
#  ["TOP_N_KEYS","10"], 
#  ["LOCK","builder","dept3"], 
#  ["LOCK","builder","dept4"], 
#  ["SET_OR_INC_BY_CALLER","dept4","floors","10","builder"], 
#  ["SET_OR_INC_BY_CALLER","dept4","floors","20","otherBuilder"], 
#  ["SET_OR_INC","dept3","floors","6"], 
#  ["SET_OR_INC_BY_CALLER","dept2","floors","7","builder"], 
#  ["GET","dept4","floors"], 
#  ["GET","dept3","floors"], 
#  ["GET","dept2","floors"], 
#  ["TOP_N_KEYS","10"]]

# ["", "", "10", "14", "6", "10", "14", "dept4(2), dept2(1), dept3(1)", "acquired", "acquired", "24", "24", "6", "17", "24", "6", "17", "dept4(3), dept2(2), dept3(1)"]

# queries:
# [["SET_OR_INC","item1","cost","42"], 
#  ["GET","item1","cost"], 
#  ["TOP_N_KEYS","1"], 
#  ["LOCK","director","item2"], 
#  ["SET_OR_INC","item2","cost","20"], 
#  ["SET_OR_INC","item2","cost","40"], 
#  ["SET_OR_INC_BY_CALLER","item2","cost","30","direct"], 
#  ["SET_OR_INC_BY_CALLER","item2","cost","12","director"], 
#  ["SET_OR_INC_BY_CALLER","item2","cost","37","director"], 
#  ["SET_OR_INC_BY_CALLER","item2","cost","51","other"], 
#  ["SET_OR_INC","item1","cost","10"], 
#  ["SET_OR_INC_BY_CALLER","item1","cost","17","director"], 
#  ["TOP_N_KEYS","1"], 
#  ["SET_OR_INC_BY_CALLER","item2","cost","38","other"], 
#  ["UNLOCK","item2"], 
#  ["SET_OR_INC_BY_CALLER","item2","cost","42","other"], 
#  ["GET","item2","cost"], 
#  ["DELETE","item1","cost"], 
#  ["TOP_N_KEYS","1"], 
#  ["DELETE","item1","cost"], 
#  ["DELETE","item2","cost"], 
#  ["GET","item2","cost"], 
#  ["TOP_N_KEYS","1"]]

# [
#   "42",
#   "42",
#   "item1(1)",
#   "invalid_request",
#   "20",
#   "60",
#   "90",
#   "102",
#   "139",
#   "190",
#   "52",
#   "69",
#   "item2(6)",
#   "228",
#   "",
#   "270",
#   "270",
#   "true",
#   "item2(8)",
#   "false",
#   "true",
#   "",
#   ""
# ]
